In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.impute import SimpleImputer

In [2]:
train=pd.read_csv(r'C:\Users\admin\Desktop\과제 모음집\flex\july-dacon\july-dacon\235745_parking_data\train.csv')
test=pd.read_csv(r'C:\Users\admin\Desktop\과제 모음집\flex\july-dacon\july-dacon\235745_parking_data\test.csv')
submission=pd.read_csv(r'C:\Users\admin\Desktop\과제 모음집\flex\july-dacon\july-dacon\235745_parking_data\sample_submission.csv')

In [3]:
train.shape, test.shape

((2952, 15), (1022, 14))

In [4]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [5]:
test.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0


In [6]:
train.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

In [7]:
test.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              2
임대보증금                           180
임대료                             180
도보 10분거리 내 지하철역 수(환승노선 수 반영)     42
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
dtype: int64

In [8]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [9]:
train.columns=['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스', '단지내주차면수', '등록차량수']

test.columns=['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스', '단지내주차면수']

In [10]:
train['지역'].unique()

array(['경상북도', '경상남도', '대전광역시', '경기도', '전라북도', '강원도', '광주광역시', '충청남도',
       '부산광역시', '제주특별자치도', '울산광역시', '충청북도', '전라남도', '대구광역시', '서울특별시',
       '세종특별자치시'], dtype=object)

In [11]:
train['공급유형'].unique()

array(['국민임대', '공공임대(50년)', '영구임대', '임대상가', '공공임대(10년)', '공공임대(분납)',
       '장기전세', '공공분양', '행복주택', '공공임대(5년)'], dtype=object)

In [12]:
local_map={} #지역, 자격유형, 공급유형, 임대건물구분 전부 숫자로 
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc]=i
train['지역']=train['지역'].map(local_map)
test['지역']=test['지역'].map(local_map)

local_type={}
for i, loc in enumerate(train['자격유형'].unique()):
    local_type[loc]=i
train['자격유형']=train['자격유형'].map(local_type)
test['자격유형']=test['자격유형'].map(local_type)

local_type2={}
for i, loc in enumerate(train['공급유형'].unique()):
    local_type2[loc]=i
train['공급유형']=train['공급유형'].map(local_type2)
test['공급유형']=test['공급유형'].map(local_type2)

local_building={}
for i, loc in enumerate(train['임대건물구분'].unique()):
    local_building[loc]=i
train['임대건물구분']=train['임대건물구분'].map(local_building)
test['임대건물구분']=test['임대건물구분'].map(local_building)

In [13]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
0,C2483,900,0,0,0,39.72,134,38.0,0,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,0,0,0,39.72,15,38.0,0,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,0,0,0,51.93,385,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,0,0,0,51.93,15,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,0,0,0,51.93,41,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0


In [14]:
train.groupby(['단지코드', '전용면적']).sum()['전용면적별세대수']

단지코드   전용면적 
C1000  39.57    344
       39.93     75
       46.70     36
       46.90     36
       51.93     75
               ... 
C2680  36.63    486
       46.71    272
C2692  36.64    334
       46.49    105
       51.59     60
Name: 전용면적별세대수, Length: 1898, dtype: int64

In [15]:
train.groupby(['단지코드']).nunique()['전용면적']

단지코드
C1000     5
C1004    15
C1005     3
C1013     4
C1014     6
         ..
C2663     5
C2666     4
C2670     4
C2680     2
C2692     3
Name: 전용면적, Length: 423, dtype: int64

In [16]:
new_train=pd.DataFrame()
new_train=train
new_train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
0,C2483,900,0,0,0,39.72,134,38.0,0,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,0,0,0,39.72,15,38.0,0,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,0,0,0,51.93,385,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,0,0,0,51.93,15,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,0,0,0,51.93,41,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,0,5,0,49.20,19,7.0,0,11346000,116090,0.0,1.0,166.0,146.0
2948,C2532,239,0,5,0,51.08,34,7.0,0,14005000,142310,0.0,1.0,166.0,146.0
2949,C2532,239,0,5,0,51.73,34,7.0,0,14005000,142310,0.0,1.0,166.0,146.0
2950,C2532,239,0,5,0,51.96,114,7.0,0,14005000,142310,0.0,1.0,166.0,146.0


In [17]:
new_train['세대수합']=new_train.groupby(['단지코드', '전용면적']).전용면적별세대수.transform('sum')
new_train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수,세대수합
0,C2483,900,0,0,0,39.72,134,38.0,0,15667000,103680,0.0,3.0,1425.0,1015.0,149
1,C2483,900,0,0,0,39.72,15,38.0,0,15667000,103680,0.0,3.0,1425.0,1015.0,149
2,C2483,900,0,0,0,51.93,385,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0,441
3,C2483,900,0,0,0,51.93,15,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0,441
4,C2483,900,0,0,0,51.93,41,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0,441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,0,5,0,49.20,19,7.0,0,11346000,116090,0.0,1.0,166.0,146.0,19
2948,C2532,239,0,5,0,51.08,34,7.0,0,14005000,142310,0.0,1.0,166.0,146.0,34
2949,C2532,239,0,5,0,51.73,34,7.0,0,14005000,142310,0.0,1.0,166.0,146.0,34
2950,C2532,239,0,5,0,51.96,114,7.0,0,14005000,142310,0.0,1.0,166.0,146.0,114


In [18]:
new_train=new_train.drop(['전용면적별세대수'], axis=1)
new_train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수,세대수합
0,C2483,900,0,0,0,39.72,38.0,0,15667000,103680,0.0,3.0,1425.0,1015.0,149
1,C2483,900,0,0,0,39.72,38.0,0,15667000,103680,0.0,3.0,1425.0,1015.0,149
2,C2483,900,0,0,0,51.93,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0,441
3,C2483,900,0,0,0,51.93,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0,441
4,C2483,900,0,0,0,51.93,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0,441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,0,5,0,49.20,7.0,0,11346000,116090,0.0,1.0,166.0,146.0,19
2948,C2532,239,0,5,0,51.08,7.0,0,14005000,142310,0.0,1.0,166.0,146.0,34
2949,C2532,239,0,5,0,51.73,7.0,0,14005000,142310,0.0,1.0,166.0,146.0,34
2950,C2532,239,0,5,0,51.96,7.0,0,14005000,142310,0.0,1.0,166.0,146.0,114


In [19]:
new_train=new_train.drop_duplicates(ignore_index=True)
new_train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수,세대수합
0,C2483,900,0,0,0,39.72,38.0,0,15667000,103680,0.0,3.0,1425.0,1015.0,149
1,C2483,900,0,0,0,51.93,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0,441
2,C2483,900,0,0,0,51.95,38.0,0,27304000,184330,0.0,3.0,1425.0,1015.0,224
3,C2483,900,0,0,0,59.88,38.0,0,30357000,214270,0.0,3.0,1425.0,1015.0,86
4,C2515,545,0,1,0,33.48,17.0,0,9216000,82940,0.0,3.0,624.0,205.0,276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1928,C2532,239,0,5,0,49.20,7.0,0,11346000,116090,0.0,1.0,166.0,146.0,19
1929,C2532,239,0,5,0,51.08,7.0,0,14005000,142310,0.0,1.0,166.0,146.0,34
1930,C2532,239,0,5,0,51.73,7.0,0,14005000,142310,0.0,1.0,166.0,146.0,34
1931,C2532,239,0,5,0,51.96,7.0,0,14005000,142310,0.0,1.0,166.0,146.0,114


In [21]:
new_train['임대보증금']=new_train['임대보증금'].replace('-',np.nan)
new_train['임대료']=new_train['임대료'].replace('-',np.nan)

<ipython-input-21-7c4544324318>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train['임대보증금']=new_train['임대보증금'].replace('-',np.nan)
<ipython-input-21-7c4544324318>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_train['임대료']=new_train['임대료'].replace('-',np.nan)


In [22]:
imputer=SimpleImputer(strategy='most_frequent')
new_train=pd.DataFrame(imputer.fit_transform(new_train))
new_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,C2483,900,0,0,0,39.72,38,0,15667000,103680,0,3,1425,1015,149
1,C2483,900,0,0,0,51.93,38,0,27304000,184330,0,3,1425,1015,441
2,C2483,900,0,0,0,51.95,38,0,27304000,184330,0,3,1425,1015,224
3,C2483,900,0,0,0,59.88,38,0,30357000,214270,0,3,1425,1015,86
4,C2515,545,0,1,0,33.48,17,0,9216000,82940,0,3,624,205,276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1928,C2532,239,0,5,0,49.2,7,0,11346000,116090,0,1,166,146,19
1929,C2532,239,0,5,0,51.08,7,0,14005000,142310,0,1,166,146,34
1930,C2532,239,0,5,0,51.73,7,0,14005000,142310,0,1,166,146,34
1931,C2532,239,0,5,0,51.96,7,0,14005000,142310,0,1,166,146,114


In [46]:
new_train.columns=['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스', '단지내주차면수', '등록차량수', '세대수합']
new_train=new_train[['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스', '단지내주차면수',  '세대수합', '등록차량수']]
new_train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,세대수합,등록차량수
0,C2483,900,0,0,0,39.72,38,0,15667000,103680,0,3,1425,149,1015
1,C2483,900,0,0,0,51.93,38,0,27304000,184330,0,3,1425,441,1015
2,C2483,900,0,0,0,51.95,38,0,27304000,184330,0,3,1425,224,1015
3,C2483,900,0,0,0,59.88,38,0,30357000,214270,0,3,1425,86,1015
4,C2515,545,0,1,0,33.48,17,0,9216000,82940,0,3,624,276,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1928,C2532,239,0,5,0,49.2,7,0,11346000,116090,0,1,166,19,146
1929,C2532,239,0,5,0,51.08,7,0,14005000,142310,0,1,166,34,146
1930,C2532,239,0,5,0,51.73,7,0,14005000,142310,0,1,166,34,146
1931,C2532,239,0,5,0,51.96,7,0,14005000,142310,0,1,166,114,146


In [24]:
new_test=pd.DataFrame()
new_test=test
new_test['세대수합']=new_test.groupby(['단지코드', '전용면적']).전용면적별세대수.transform('sum')
new_test=new_test.drop(['전용면적별세대수'], axis=1)
new_test=new_test.drop_duplicates(ignore_index=True)

new_test['임대보증금']=new_test['임대보증금'].replace('-',np.nan)
new_test['임대료']=new_test['임대료'].replace('-',np.nan)

imputer=SimpleImputer(strategy='most_frequent')
new_test=pd.DataFrame(imputer.fit_transform(new_test))

new_test.columns=['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '공가수',
       '자격유형', '임대보증금', '임대료', '지하철', '버스', '단지내주차면수', '세대수합']
new_test

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,세대수합
0,C1072,754,0,3,0,39.79,14,7,22830000,189840,0,2,683,116
1,C1072,754,0,3,0,46.81,14,0,36048000,249930,0,2,683,30
2,C1072,754,0,3,0,46.9,14,7,36048000,249930,0,2,683,232
3,C1072,754,0,3,0,51.46,14,7,43497000,296780,0,2,683,60
4,C1072,754,0,3,0,51.71,14,7,43497000,296780,0,2,683,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,C1267,675,0,1,8,26.85,38,11,25600000,187260,0,1,467,74
709,C1267,675,0,1,8,36.77,38,11,25600000,187260,0,1,467,126
710,C2189,382,0,4,0,29.19,45,7,6872000,106400,0,2,300,116
711,C2189,382,0,4,0,39.45,45,7,13410000,144600,0,2,300,202


In [62]:
x_train = new_train.iloc[:, 1:-1]
y_train = new_train.iloc[:,-2]
x_test = new_test.iloc[:,1:]

In [48]:
x_train=x_train.apply(pd.to_numeric)
x_train.info()

x_test=x_test.apply(pd.to_numeric)
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1933 entries, 0 to 1932
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   총세대수     1933 non-null   int64  
 1   임대건물구분   1933 non-null   int64  
 2   지역       1933 non-null   int64  
 3   공급유형     1933 non-null   int64  
 4   전용면적     1933 non-null   float64
 5   공가수      1933 non-null   float64
 6   자격유형     1933 non-null   int64  
 7   임대보증금    1933 non-null   int64  
 8   임대료      1933 non-null   int64  
 9   지하철      1933 non-null   float64
 10  버스       1933 non-null   float64
 11  단지내주차면수  1933 non-null   float64
 12  세대수합     1933 non-null   int64  
dtypes: float64(5), int64(8)
memory usage: 196.4 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 713 entries, 0 to 712
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   총세대수     713 non-null    int64  
 1   임대건물구분   713 non-null    int64  
 2   지역       

# 예측 과정

In [26]:
from sklearn.linear_model import LinearRegression

In [49]:
x_test

,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,세대수합
0,754,0,3,0,39.79,14.0,7.0,22830000,189840,0.0,2.0,683.0,116
1,754,0,3,0,46.81,14.0,0.0,36048000,249930,0.0,2.0,683.0,30
2,754,0,3,0,46.90,14.0,7.0,36048000,249930,0.0,2.0,683.0,232
3,754,0,3,0,51.46,14.0,7.0,43497000,296780,0.0,2.0,683.0,60
4,754,0,3,0,51.71,14.0,7.0,43497000,296780,0.0,2.0,683.0,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,675,0,1,8,26.85,38.0,11.0,25600000,187260,0.0,1.0,467.0,74
709,675,0,1,8,36.77,38.0,11.0,25600000,187260,0.0,1.0,467.0,126
710,382,0,4,0,29.19,45.0,7.0,6872000,106400,0.0,2.0,300.0,116
711,382,0,4,0,39.45,45.0,7.0,13410000,144600,0.0,2.0,300.0,202


In [124]:
results=[]
x_train = new_train.iloc[:,1:-1]
y_train = new_train.iloc[:,-1]
model.fit(x_train, y_train)

for (i, data_grouped) in new_test.groupby(['단지코드']):
    results.append(model.predict(data_grouped.iloc[:,1:])[0])
    groups.append(i)
results

[296.0,
 250.62,
 718.64,
 330.5,
 29.41,
 351.21,
 530.88,
 579.52,
 816.75,
 441.36,
 673.18,
 368.89,
 987.29,
 1284.08,
 229.22,
 831.08,
 529.71,
 926.44,
 294.75,
 301.56,
 316.94,
 727.42,
 331.61,
 147.27,
 214.75,
 423.31,
 599.99,
 877.44,
 517.23,
 455.57,
 555.66,
 405.08,
 480.3,
 402.62,
 317.06,
 216.67,
 410.27,
 88.64,
 958.62,
 267.5,
 418.93,
 670.9,
 382.31,
 734.89,
 495.9,
 510.74,
 472.44,
 127.93,
 864.61,
 1082.27,
 632.48,
 912.2,
 1068.9,
 1538.82,
 251.31,
 954.73,
 259.39,
 282.75,
 340.06,
 488.57,
 918.92,
 572.12,
 280.62,
 127.05,
 174.51,
 377.73,
 386.31,
 267.65,
 1065.43,
 263.99,
 624.21,
 557.89,
 84.67,
 447.94,
 524.47,
 370.75,
 421.35,
 911.77,
 111.75,
 238.94,
 427.48,
 792.87,
 928.06,
 569.79,
 77.79,
 515.72,
 719.36,
 747.33,
 317.5,
 418.26,
 805.04,
 989.24,
 167.37,
 797.33,
 255.82,
 305.7,
 1024.9,
 159.6,
 392.07,
 532.03,
 294.19,
 546.14,
 581.64,
 34.87,
 473.24,
 901.6,
 204.11,
 239.02,
 407.52,
 438.57,
 611.25,
 123.7,
 1150

In [125]:
len(results)

150

In [127]:
submission['num'] = results

In [129]:
submission.to_csv('제출용.csv', index=False)